# Простейшие нейронные сети с Keras


Фреймворк Keras+Tensorflow сейчас стремительно теряет популярность, но еще совсем недавно многие сетки писались именно на нем, поэтому какое-то базовое знакомство все равно пригодится. Сейчас актуальна версия TF номер 2.

Одна из особенностей новой версии заключается в том, что Keras (раньше он был посторонней библиотекой-надстройкой) фактически [стал частью tensorflow.](https://www.tensorflow.org/guide/keras?hl=ru)  

Изначально Keras создавался как высокоуровневое API для theano (это когда написали кучу функций один раз, красиво их назвали и завернули в пакет). Потом он начал поддерживать tensorflow, а сейчас вот чуть ли ни аннексия произошла. __Keras наш.__ Именно со знакомства с ним мы и начнём наше погружение в нейросетки.  

Чтобы установить библиотеку, отправляйтесь в консоль и пропишите 

```
pip install tensorflow
```

Если вы до этого сидели на старой версии tensorflow, на всякий случай снесите её. Иначе при установке могут возникнуть какие-нибудь несовместимости и ошибки.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Подгружаем tensorflow 
import tensorflow as tf
import tensorflow.keras as keras


print(tf.__version__)


In [ ]:
# Подгрузим ещё немного пакетов :) 
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

## 1. О данных и бейзлайнах

Наша главная цель - как следует потрогать Keras своими руками. Мы хотим увидеть, что он реально позволяет обращаться с нейронками как с конструктором LEGO и предлагает нам для сборки башен кучу деталей. 

__Делать всё это мы будем на животных.__ Ежегодно около 7.6 миллионов бедных животных в США оказываются в приютах. Часть из них находит себе новую семью, часть возвращается к старому (бывает, что питомец потерялся и его нашли на улице), а часть погибает. Ужегодно усыпляется около 2.7 млн. собак и кошек.  

Используя датасет с входной информацией (цвет, пол, возраст и т.п.) из одного из приютов, мы попытаемся спрогнозировать что произойдёт с новыми животными, которые попадут в этот приют. Данные, используемые в тетрадке уже были предварительно обработаны и приведены в удобную для построения моделей форму. 

In [ ]:
X = pd.read_csv('data/X_cat.csv', sep = '\t', index_col=0)
target = pd.read_csv('data/y_cat.csv', sep = '\t', index_col=0, names=['status'])

print(X.shape)
X.head()

В датасете находится около 27 тысяч наблюдений и 39 регрессоров. Посмотрим на то как выглядит распределение того, что произошло со зверятами по особям.

In [ ]:
target.status.value_counts()

Видим, что классы несбалансированы. Попробуем оставить четыре класса и объединить класс умерших животных с классом животных, которых усыпили. 

In [ ]:
target = target.values
target[target == 'Died'] = 'Euthanasia'

Закодируем классы.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(target)
y

In [ ]:
len(y)

In [ ]:
X.shape

Разобьём выборку на тренировочную и тестовую. 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state = 42)

print(X_train.shape)
print(X_test.shape)

Прежде, чем учить 228-слойных монстров, давайте построим какие-нибудь простые прогнозы, чтобы было с чем сравнить. Давайте построим наивный прогноз, а также обучим линейную регрессию и случайный лес.

### Константный прогноз

Построим константный прогноз, чтобы было с чем сравнивать и прогноз по какой-нибудь модели. 

In [ ]:
from sklearn.metrics import log_loss

probas = np.array(pd.Series(y_train).value_counts(normalize=True).sort_index().tolist())
probas

In [ ]:
log_loss(y_test, np.tile(probas, X_test.shape[0]).reshape(X_test.shape[0], 4))

### Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression

# обучите логистическую регрессию с предсказанием вероятностей (predict_proba)

In [ ]:
# выведите предсказания модели на тесте

### Случайный лес 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# обучите рандомный лес и посчитайте logloss 

Попробуем улучшить результат с помощью нейросеток. 

## 2. Собираем свою нейросеть

In [ ]:
from tensorflow.keras import layers as L         # Уже готовые слои для моделей
from tensorflow.keras.models import Sequential   # Специальный класс для склеивания слоёв
from tensorflow.keras.models import Model        # Альтернативный класс для склейки слоёв
import tensorflow.keras.optimizers as opt        # Разные оптимизационные алгоритмы :3 

Модель в Keras собирается как конструктор LEGO. Её можно описать двумя основными способами. Первый — последовательное описание модели. В его случае мы как бы создаём коробочку `model` и постепенно добавляем туда детальки нашей сетки. 

Второй способ - описать модель функционально, в явном виде прописав какие аргументы идут на вход какому слою. 

Когда мы будем строить сложные модели с кучей разветвлений, для нас будет удобным второй способ. Для простых ситуаций вроде текущей, можете пользоваться первым. 

Давайте соберём небольшую нейронку. Давайте сделаем в ней вот такую архитектуру: 

In [ ]:
from tensorflow.keras.metrics import Accuracy

def get_new_model():
    acc = Accuracy()

    ###########################################################
    model = Sequential(name = 'Archibald')  # модели можно дать имя!
    
    # Добавляем в нашу модель первый слой из 25 нейронов
    model.add(L.Dense(25, input_dim = X_train.shape[1], kernel_initializer='random_normal'))

    # Добавляем функцию активации на первый слой 
    model.add(L.Activation('sigmoid'))

    # Добавляем ещё один слой из 25 нейронов
    model.add(L.Dense(25, kernel_initializer='random_normal'))
    model.add(L.Activation('sigmoid'))

    # На выходе мы должны получить вероятности того, что объект относится к разным классам 
    # Сделать такое преобразование позволяет softmax как функция активации
    # На выход будет идти 4 вероятности по числу классов
    model.add(L.Dense(4, activation='softmax', kernel_initializer = 'random_normal'))
    
    ###########################################################
    
    # В качестве оптимизации будем использовать Adam
    # Это такой специальный градиентный спуск, обсудим его в следущий раз
    optimizer = opt.Adam(lr=1e-3)

    # Собираем модель
    model.compile(loss = 'categorical_crossentropy', 
                  metrics=['acc'], 
                  optimizer=optimizer)
    
    return model

Перед тем как обучать нашу нейросеть, нужно задать параметры обучения. Во-первых, метод оптимизации. Во-вторых, функцию потерь. В-третьих, парочку метрик, на которые нам хотелось бы смотреть в процессе обучения.  Для этого есть метод `compile`. 

In [ ]:
model = get_new_model()

Итак, только что мы собрали свою первую нейросеть со скрытым слоем.   Посмотрим сколько параметров нам предстоит оценить. 

In [ ]:
model.summary()

Видим, что нужно оценить огромное количество параметров. На оценку каждого параметра придётся совсем маленькое количество наблюдений. 

In [ ]:
X_train.shape[0] / 1704

Отлично! Модель собрана. Осталось её обучить. Метод `fit` позволяет сделать это. Он принимает на вход обучающую выборку `X_train, y_train`. Параметр  `batch_size` это  размер батча. Этот параметр ограничивает количество примеров, которое за раз подаётся в нашу нейронную сеть. Количиство эпох для обучения, `epochs` - это число проходов модели по обучающей выборке. 

Обычно нейросетки обучаются довольно долго. В связи с этим обучать их на различных фолдах и оценивать качество работы модели на кросс-валидации не очень быстрое занятие, которое себе может позволить далеко не каждый. Для проверки качества обычно внутри обучения с помощью параметра `validation_split` часть выборки оставляют под валидацию, которая осуществляется после каждой эпохи. Как только качество на валидации начнёт падать, это будет означать, что модель переобучилаcь. 

Всё, что будет нам возвращать метод `.fit`, мы запишем в переменную `hist`. После обученя модели там окажется вся история ошибок на каждом шаге обучения. Отключить сообщения о том, на каком этапе обучения в данный момент находится модель, можно с параметром `verbose = 0`. Переменную $y$ для успешного обучения сетки нужно будет перевести в матрицу из дамми-переменных с помощью команды `to_categorical`. 

In [ ]:
from tensorflow.keras.utils import to_categorical
to_categorical(y)

In [ ]:
hist = model.fit(X_train, to_categorical(y_train), validation_split=0.2, epochs=30, verbose=1, batch_size=128)

In [ ]:
preds = tf.argmax(model.predict(X_train), axis=1)

In [ ]:
preds

In [ ]:
acc = Accuracy()

In [ ]:
acc(preds,y_train)

Возвращает этот метод history — это история ошибок на каждом шаге обучения. Посмотрим на динамику ошибок нашей модели во времени. 

In [ ]:
start = 1
plt.plot(hist.history['loss'][start:])
plt.plot(hist.history['val_loss'][start:])
plt.legend(['Train loss', 'Validation loss'])

In [ ]:
# Можно предсказать вероятности для тестовой выборки
model.predict(X_test)

In [ ]:
# А можно классы 
tf.argmax(model.predict(X_train), axis=1)

In [ ]:
# Можно найти logloss
log_loss(y_test, model.predict(X_test))

In [ ]:
# Можно сохранить модель
model.save("weights.h5")

In [ ]:
# Можно посмотреть на качество модели по всем указанным при коммпиляции метрикам. 
score = model.evaluate(X_test, to_categorical(y_test), verbose=0)
score

Модель сходится довольно плохо. Функция потерь на валидации со временем начинает скакать. __Как думаете, с чем это связано?__

----------------------------------

__Ответ:__ Вспомним про такую замечательную штуку как нормализация и посмотрим насколько она улучшит скорость обучения и качество модели.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Обучаем модель на нормализованных данных. Попробуем учить подольше. Так сказать: двойной удар по сходимости. 

In [ ]:
model = get_new_model()  # Эпох стало 300 вместо 100
hist = model.fit(X_train, to_categorical(y_train), validation_split=0.2, epochs=30, verbose=0)

In [ ]:
start = 0
plt.plot(hist.history['loss'][start:])
plt.plot(hist.history['val_loss'][start:])
plt.legend(['Train loss', 'Validation loss'])
plt.show()

In [ ]:
score = model.evaluate(X_test, to_categorical(y_test), verbose=0)
score 

Качество нашей модели на валидационных данных достигает пика после определенного количества эпох, а затем начинает снижаться. Это происходит из-за того, что модель начинает __переобучаться.__ Очень важно знать способы как можно предотвратить это. Переобученая модель просто запоминает выборку. Нам бы хотелось, чтобы она обобщала закономерности и адекватно вела себа на новых данных.

Обратным случаем переобучения является __недообучение.__ Оно возникает, когда все еще есть возможность улучшить показатели модели на проверочном наборе данных. Недообучение может произойти по разным причинам: например, если модель недостаточно сильная, или слишком сложная, или просто недостаточно тренировалась на данных. В любом случае это будет означать, что не были выучены основные паттерны из проверочного сета. Нам нужно найти золотую середину.

Чтобы избежать переобучения, наиболее оптимальным решением будет использовать больше тренировочных данных. Модели, обученные на большем количестве данных, естественным образом обобщают их лучше. Если возможность раздобыть ещё данных исчерпана, можно использовать методы регуляризации. Они ограничивают количество и тип инофрмации, которые модель может хранить в себе. Если нейросеть может запомнить только небольшое количество паттернов, то тогда процесс оптимизации заставит ее сфокусироваться на самых важных, наиболее заметных шаблонах, которые будут иметь более высокий шанс обобщения. __Попробуем дать переобучению бой!__

## Различные воспитательные трюки 

1. Начните с маленькой сети. Не забывайте прикидывать сколько наблдюдений $n$ тратится на оценку каждого из $k$ параметров. Если величина $\frac{n}{k}$ очень маленькая, то не может идти речи об адекватных оценках параметров!  
2. Всегда оставляйте часть выборки под валидацию на каждой эпохе.
3. Усложняйте модель, пока качество на валидации не начнёт падать.
4. Не забывайте проскалировать ваши наблюдения для лучшей сходимости. 
5. Можно попробовать ещё целую серию различных **трюков**: 


* __Архитектура нейросети__
    * Больше/меньше нейронов
    * Больше/меньше слоёв 
    * Другие функции активации (tanh, relu, leaky relu, elu etc) 
    * Регуляризация (dropout, l1,l2) 


* __Более качественная оптимизация__  
    * Можно попробовать выбрать другой метод оптимизации 
    * Можно попробовать менять скорость обучения, моментум и др.
    * Разные начальные значения весов


* __Попробовать собрать больше данных__ 
* __Для случая картинок объёмы данных можно увеличить искусственно с помощью подхода, который называется Data augmemntation, но об этом позже__

И это далеко не полный список. Обратите внимание, что делать grid_search для больших сеток это довольно времязатратное занятие... 

## 4. Эксперименты 

Наверняка, у каждого в голове возникла какая-то классная идея по улучшению нашей базовой архитектуры. Пора реализовать её! Если идей нет, попробуйте: 

- Поменять функции активации на elu или сигмоиды
- Увеличить размер архитектуры 

Не забывайте сохранять историю обучения и закидывать её на наш график для сравнения. Когда надоест, посмотрите на итоговое качество модели на тесте. А то по валидации и переобучиться можно :) 

In [ ]:
def get_new_model():
    acc = Accuracy()

    model = Sequential([
        L.Dense(25, input_dim = X_train.shape[1], kernel_initializer='he_normal'),
        L.Dropout(0.2),
        L.Activation(tf.nn.relu),
        L.Dense(25, kernel_initializer='he_normal', activation='ReLU'),
        L.Dense(4, activation='softmax', kernel_initializer='random_normal')

    ], name = 'First')  

    optimizer = opt.Adam(lr=1e-3)

    model.compile(loss = 'categorical_crossentropy', 
                  metrics=['acc'], 
                  optimizer=optimizer)
    
    return model

In [ ]:
model = get_new_model()

In [ ]:
hist = model.fit(X_train, to_categorical(y_train), validation_split=0.2, epochs=30, verbose=1, batch_size=128)

In [ ]:
score = model.evaluate(X_test, to_categorical(y_test), verbose=0)
score 

In [ ]:
start = 0
plt.plot(hist.history['loss'][start:])
plt.plot(hist.history['val_loss'][start:])
plt.legend(['Train loss', 'Validation loss'])